#**AutoScratchAI**
 ## A Prototype for Automated Vehicle Scratch Detection
### 1. Overview

This project focuses on developing a robust computer vision system for **vehicle scratch detection** using computer vision techniques. It is intended for deployment in an **insurtech workflow**, where quick and accurate identification of vehicle damage (scratches) from images can streamline insurance claims and assessments.

 ### 1.1 Objectives

- Develop a high-performing deep learning model to **detect visible scratches** on vehicle surfaces.
- Build a **YOLOv8-based object detection pipeline** using custom labeled data.
- Integrate the trained model into a **Flask web application** for real-time scratch detection.
- Ensure the solution is **scalable, lightweight**, and aligns with practical deployment use cases in **digital claims and underwriting workflows**.
## 2. Data Collection

### 2.1 Data Sources

- **CarDD Dataset**\
  └─ Includes images with vehicle scratches and corresponding COCO-format annotations (train/val).

- **Stanford Cars Dataset**\
  └─ Used to source "no\_damage or scratch" vehicle images collected from ["https://www.kaggle.com/cyizhuo/stanford-cars-by-classes-folder]("https://www.kaggle.com/cyizhuo/stanford-cars-by-classes-folder).

### 2.2 License & Provenance

All datasets were sourced from public repositories or academic datasets, with appropriate usage rights and citations maintained.


In [ ]:
# Mount Google Drive to access dataset and save outputs in Colab environment
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#  Navigate to your project directory mounted on Google Drive
%cd /content/drive/MyDrive/vehicle_scratch_detection_v2






In [ ]:
# Install required libraries
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu118
!pip install pycocotools
!pip install opendatasets
!pip install ultralytics


In [ ]:
# Print details about the installed PyTorch version and GPU support status.
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device")


Torch version: 2.6.0+cu124
CUDA available: False
CUDA device name: No CUDA device


To implement the project in google colab using Python, the required libraries were imported.

In [ ]:
# import required libraries
import json
import matplotlib.pyplot as plt
from collections import Counter
import shutil, random
import os
import torch
import time
import ultralytics
ultralytics.checks()
import opendatasets as od
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split
from PIL import Image
from ultralytics.data.converter import convert_coco
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path




## Data Collection


**CarDD Dataset (Scratched Vehicles)**
Source: CarDD on Roboflow

The data contain labeled images of vehicles with scratches, dents, and damages in COCO format. The ZIP file was manually downloaded from Roboflow and uploaded to Google Drive.

Unzipping CarDD Dataset

In [ ]:
!unzip -q "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/CarDD/CarDD_release.zip" \
  -d "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/CarDD/"

Directory Structure Check

In [ ]:
!ls "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/CarDD/"
!ls "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/CarDD/CarDD_release"
!ls "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/CarDD/CarDD_release/CarDD_COCO/"



Stanford Cars Dataset (Clean Vehicles)

Source: Stanford Cars by Classes Folder on Kaggle.

Images were used to represent no-scratch (negative class) examples. The download was done via the opendatasets library directly into Google Drive.

Download and unzip Stanford Cars Dataset

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/cyizhuo/stanford-cars-by-classes-folder",
            data_dir="/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars")

Skipping, found downloaded files in "/content/drive/MyDrive/Vehicle_Scratch_Detection_Model/Dataset/stanford_cars/stanford-cars-by-classes-folder" (use force=True to force download)


## Data Exploration

This section explores the class distribution in the CarDD dataset (used for damage/scratch images) and prepares a curated subset of clean car images from the Stanford Cars dataset to serve as the no-damage class.



1. Inspect CarDD Annotations

We begin by inspecting the available categories, number of images, and annotation counts in the COCO-formatted CarDD dataset:

Dataset Overview and Sample Annotations

In [ ]:
# Load COCO annotations
json_path = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations/instances_train2017.json"
with open(json_path, "r") as f:
    coco_data = json.load(f)

# Basic dataset info
print("CarDD Train Dataset Info")
print("Categories:", [c['name'] for c in coco_data['categories']])
print("Number of images:", len(coco_data['images']))
print("Number of annotations:", len(coco_data['annotations']))

# Show sample annotations
print("\nSample Annotations:")
for ann in coco_data['annotations'][:5]:
    print(f"  ID: {ann['id']}, Image ID: {ann['image_id']}, Category ID: {ann['category_id']}, BBox: {ann['bbox']}")


In [ ]:
# Load COCO annotations for test set
val_json_path = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations/instances_val2017.json"
with open(test_json_path, "r") as f:
    val_data = json.load(f)

# Basic Val dataset info
print("CarDD Test Dataset Info")
print("Categories:", [c['name'] for c in val_data['categories']])
print("Number of images:", len(val_data['images']))
print("Number of annotations:", len(val_data['annotations']))

# Show sample annotations
print("\nSample Annotations (Test Set):")
for ann in val_data['annotations'][:5]:
    print(f"  ID: {ann['id']}, Image ID: {ann['image_id']}, Category ID: {ann['category_id']}, BBox: {ann['bbox']}")

Visualize Category Distribution

In [ ]:
# Count frequency of each category ID in annotations
category_counts = Counter([ann['category_id'] for ann in coco_data['annotations']])

# Map category IDs to names
id_to_name = {cat['id']: cat['name'] for cat in coco_data['categories']}
category_names = [id_to_name[cid] for cid in category_counts.keys()]
frequencies = list(category_counts.values())

# Plot
plt.figure(figsize=(8, 5))
plt.bar(category_names, frequencies, color="skyblue", edgecolor="black")
plt.title("Category Distribution in CarDD Dataset")
plt.xlabel("Category")
plt.ylabel("Number of Annotations")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


2. Prepare No-Damage (Clean Vehicle) Images from Stanford Cars Dataset

We extracted clean vehicle images from the Stanford Cars Dataset to represent the negative class (no visible damage). Each image is assigned an empty YOLO .txt label file.

In [ ]:

# Define source and target directories
source_dir = '/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/stanford-cars-by-classes-folder/train'
target_image_dir = '/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/images/all'
target_label_dir = '/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/labels/all'

# Create dirs
os.makedirs(target_image_dir, exist_ok=True)
os.makedirs(target_label_dir, exist_ok=True)

# Collect all image paths
valid_exts = ('.jpg', '.jpeg', '.png')
all_images = [
    os.path.join(dp, f)
    for dp, dn, files in os.walk(source_dir)
    for f in files if f.lower().endswith(valid_exts)
]
print(f"Total valid images found in Stanford Cars: {len(all_images)}")

# Sample up to 2000 images
sample_size = min(2000, len(all_images))
sampled = random.sample(all_images, sample_size)

# Copy images and create empty labels
for i, src_path in enumerate(sampled):
    ext = os.path.splitext(src_path)[1].lower()
    img_name = f'{i:04d}{ext}'
    label_name = f'{i:04d}.txt'

    img_dest = os.path.join(target_image_dir, img_name)
    lbl_dest = os.path.join(target_label_dir, label_name)

    shutil.copy2(src_path, img_dest)
    with open(lbl_dest, 'w') as f:
        pass


    if (i + 1) % 200 == 0 or (i + 1) == sample_size:
        print(f"Processed {i + 1}/{sample_size} images and labels...")

print(f"\nAll done!")
print(f"  Images saved to: {target_image_dir}")
print(f"  Labels saved to: {target_label_dir}")


📸 Total valid images found in Stanford Cars: 8144
Processed 200/2000 images and labels...
Processed 400/2000 images and labels...
Processed 600/2000 images and labels...
Processed 800/2000 images and labels...
Processed 1000/2000 images and labels...
Processed 1200/2000 images and labels...
Processed 1400/2000 images and labels...
Processed 1600/2000 images and labels...
Processed 1800/2000 images and labels...
Processed 2000/2000 images and labels...

✅ All done!
  Images saved to: /content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/images/all
  Labels saved to: /content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/labels/all


3. Split No-Damage Data into Training & Validation Sets

We split the clean vehicle images into training and validation sets (80/20 split) while preserving image-label alignment.

In [ ]:
# define a function to split the data into train and val set
def split_dataset(image_dir, label_dir, output_base, train_ratio=0.8, seed=42):
    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

    train_files, val_files = train_test_split(image_files, train_size=train_ratio, random_state=seed)

    # Define output dirs
    output_dirs = {
        'train_img': os.path.join(output_base, 'images/train'),
        'val_img': os.path.join(output_base, 'images/val'),
        'train_lbl': os.path.join(output_base, 'labels/train'),
        'val_lbl': os.path.join(output_base, 'labels/val')
    }
    for dir_path in output_dirs.values():
        os.makedirs(dir_path, exist_ok=True)

    def copy_files(file_list, img_dest, lbl_dest):
        for img_file in file_list:
            base_name = os.path.splitext(img_file)[0]
            label_file = base_name + '.txt'

            # Source paths
            img_src = os.path.join(image_dir, img_file)
            lbl_src = os.path.join(label_dir, label_file)

            # Destination paths
            shutil.copy2(img_src, os.path.join(img_dest, img_file))
            if os.path.exists(lbl_src):
                shutil.copy2(lbl_src, os.path.join(lbl_dest, label_file))

    copy_files(train_files, output_dirs['train_img'], output_dirs['train_lbl'])
    copy_files(val_files, output_dirs['val_img'], output_dirs['val_lbl'])

    print(f" Split complete:")
    print(f"Training: {len(train_files)} images")
    print(f" Validation: {len(val_files)} images")



In [ ]:
# Apply the function the data set
split_dataset(
    image_dir='/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/images/all',
    label_dir='/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/labels/all',
    output_base='/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset'
)


 Split complete:
  → Training: 1600 images
  → Validation: 400 images


## Data Preprocessing / Transformation

In this step, we prepare our dataset by **unifying all damage-related annotations** into a **single class: `"scratch"`**, which will be used to train a binary object detection model. The dataset follows the COCO format, so we modify its annotations accordingly.


1. Define a function to map all annotations to class `"scratch"`

We create a utility function to **remap all existing annotation categories** to a single class with:
- `id = 0`
- `name = "scratch"`

This ensures that our object detection model learns to detect **any kind of visible vehicle damage** (e.g., dents, cracks, broken lamps, etc.) as a **binary scratch/no-scratch problem** as defined by the project objective.

In [ ]:
# define a function to map all annotations to "scratch"
def map_all_classes_to_scratch(input_json_path, output_json_path):
    """
    Remap all categories in a COCO annotation JSON to a single category 'scratch' (ID=0).

    Args:
        input_json_path (str): Path to the original COCO annotation file.
        output_json_path (str): Path to save the modified COCO annotation file.
    """
    # Load the original COCO annotations
    with open(input_json_path, 'r') as f:
        data = json.load(f)

    # Set a single category: 'scratch' with ID 0
    data['categories'] = [{'id': 0, 'name': 'scratch'}]

    # Remap all annotation category_ids to 0
    for ann in data['annotations']:
        ann['category_id'] = 0

    # Save the modified annotations
    os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
    with open(output_json_path, 'w') as f:
        json.dump(data, f)

    print(f"All categories mapped to 'scratch' and saved to: {output_json_path}")


2. Apply the mapping to both training and validation sets
We run the function on both `instances_train2017.json` and `instances_val2017.json` to generate new, unified annotation files:

In [ ]:
#Unify all damage classes in the carDD to map all damage classes as scratch
map_all_classes_to_scratch(
  "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations/instances_train2017.json",
  "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/train2017_unify.json"
)
map_all_classes_to_scratch(
  "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations/instances_val2017.json",
  "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/val2017_unify.json"
)

✅ All categories mapped to 'scratch' and saved to: /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/train2017_unify.json
✅ All categories mapped to 'scratch' and saved to: /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/val2017_unify.json


3. Define a verification function to confirm the mapping
This function checks that:

There is only one category: scratch

All annotations have `category_id = 0`

In [ ]:
# define a function that confirms that only class ID 0 is present
def verify_scratch_mapping(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Print category info
    print("Categories in JSON:")
    for cat in data['categories']:
        print(f"  ID: {cat['id']}  Name: {cat['name']}")

    # Count category_id occurrences in annotations
    category_counts = {}
    for ann in data['annotations']:
        cid = ann['category_id']
        category_counts[cid] = category_counts.get(cid, 0) + 1

    print("\nAnnotation Category ID Counts:")
    for cid, count in category_counts.items():
        print(f"  Class ID {cid}: {count} annotations")

    # Check if only one class ID exists and it's 0
    if list(category_counts.keys()) == [0]:
        print("\nAll annotations successfully mapped to class ID 0 (scratch).")
    else:
        print("\nFound unexpected class IDs:", category_counts.keys())



4. Run verification on both datasets

In [ ]:
# check the counts on train and val annotations files
verify_scratch_mapping("/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/train2017_unify.json")
verify_scratch_mapping("/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/val2017_unify.json")


📁 Categories in JSON:
  ID: 0  Name: scratch

📝 Annotation Category ID Counts:
  Class ID 0: 6211 annotations

✅ All annotations successfully mapped to class ID 0 (scratch).


Convert Unified COCO to YOLO Format (Scratch-Only Annotations)

This step converts the unified **COCO-format annotation files** into YOLO-format `.txt` files for object detection training.

We use a **custom function** because the default Ultralytics COCO converter gave incorrect results (e.g., `-1` category IDs), especially for scratch-only cases.

In [ ]:
# define a function that runs the custom conversion of coco to yolo format on your train and validation sets.
# copies the image
def convert_coco_to_yolo_scratch_only(
    json_path,
    output_dir,
    image_source_dir=None,
    image_dir_name="images",
    label_dir_name="labels"
):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Create lookup for images
    images = {img['id']: img for img in data['images']}
    annotations = data['annotations']

    # Determine which split
    split = "train" if "train" in os.path.basename(json_path) else "val"

    # Prepare output directories
    label_split_dir = os.path.join(output_dir, label_dir_name, split)
    image_split_dir = os.path.join(output_dir, image_dir_name, split)
    os.makedirs(label_split_dir, exist_ok=True)
    os.makedirs(image_split_dir, exist_ok=True)

    # Group annotations per image
    label_data = defaultdict(list)
    invalid_count = 0

    for ann in tqdm(annotations, desc=f"Converting {split} annotations"):
        image_id = ann["image_id"]
        category_id = ann["category_id"]

        if category_id != 0:
            invalid_count += 1
            continue

        image_info = images[image_id]
        width, height = image_info["width"], image_info["height"]
        bbox = ann["bbox"]  # [x_min, y_min, width, height]

        # Convert to YOLO format
        x_center = (bbox[0] + bbox[2] / 2) / width
        y_center = (bbox[1] + bbox[3] / 2) / height
        w_norm = bbox[2] / width
        h_norm = bbox[3] / height
        yolo_line = f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"

        label_data[image_info["file_name"]].append(yolo_line)

    # Write all YOLO annotations
    for image_filename, yolo_lines in label_data.items():
        label_filename = os.path.splitext(image_filename)[0] + ".txt"
        label_path = os.path.join(label_split_dir, label_filename)

        with open(label_path, "w") as f:
            f.write("\n".join(yolo_lines) + "\n")

        # copy image file
        if image_source_dir:
            src_image_path = os.path.join(image_source_dir, split, image_filename)
            dst_image_path = os.path.join(image_split_dir, image_filename)
            if os.path.exists(src_image_path):
                shutil.copy2(src_image_path, dst_image_path)
            else:
                print(f"Image file not found: {src_image_path}")

    print(f"\nConversion complete for {split} split.")
    print(f"YOLO labels saved in: {label_split_dir}")
    print(f"Images copied to: {image_split_dir}" if image_source_dir else "")
    print(f"Skipped annotations with non-zero category_id: {invalid_count}")


In [ ]:
train_json = "/path/to/train2017_unify_scratch_only.json"
val_json   = "/path/to/val2017_unify_scratch_only.json"
image_src  = "/path/to/CarDD/images"  # e.g. contains /train/ and /val/

output_dir_base = "/path/to/output/yolo_scratch_only"

# Convert
convert_coco_to_yolo_scratch_only(train_json, output_dir_base, image_source_dir=image_src)
convert_coco_to_yolo_scratch_only(val_json, output_dir_base, image_source_dir=image_src)


In [ ]:

def convert_coco_to_yolo_scratch_only(json_path, output_dir, image_dir_name="images", label_dir_name="labels"):
    with open(json_path, 'r') as f:
        data = json.load(f)

    images = {img['id']: img for img in data['images']}
    annotations = data['annotations']

    # Output folders
    label_output_dir = os.path.join(output_dir, label_dir_name)
    image_output_dir = os.path.join(output_dir, image_dir_name)

    os.makedirs(label_output_dir + "/train", exist_ok=True)
    os.makedirs(label_output_dir + "/val", exist_ok=True)
    os.makedirs(image_output_dir + "/train", exist_ok=True)
    os.makedirs(image_output_dir + "/val", exist_ok=True)

    # Determine which split
    split = "train" if "train" in os.path.basename(json_path) else "val"

    invalid_count = 0
    for ann in tqdm(annotations, desc=f"Converting {split} annotations"):
        image_id = ann["image_id"]
        category_id = ann["category_id"]
        if category_id != 0:
            # We expect all to be mapped to scratch (ID 0)
            invalid_count += 1
            continue

        image_info = images[image_id]
        width = image_info["width"]
        height = image_info["height"]
        image_filename = image_info["file_name"]
        label_filename = os.path.splitext(image_filename)[0] + ".txt"

        bbox = ann["bbox"]  # [x_min, y_min, width, height]
        x_center = (bbox[0] + bbox[2] / 2) / width
        y_center = (bbox[1] + bbox[3] / 2) / height
        w_norm = bbox[2] / width
        h_norm = bbox[3] / height

        yolo_line = f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n"

        label_path = os.path.join(label_output_dir, split, label_filename)
        with open(label_path, "a") as f:
            f.write(yolo_line)

    print(f"✅ Conversion complete. Invalid/Skipped annotations: {invalid_count}")
    print(f"📝 Labels saved to: {label_output_dir}/{split}/")



In [ ]:
train_json = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/train2017_unify_scratch_only.json"
val_json   = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/val2017_unify_scratch_only.json"
image_src  = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO"

output_dir_base = "/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/yolo_scratch_only"

# Train and Val split dirs
train_output = os.path.join(output_dir_base, "train")
val_output   = os.path.join(output_dir_base, "val")

# Convert both sets
convert_coco_to_yolo_scratch_only(train_json, output_dir_base)
convert_coco_to_yolo_scratch_only(val_json, output_dir_base)

Converting train annotations: 100%|██████████| 6211/6211 [00:44<00:00, 139.89it/s]

✅ Conversion complete. Invalid/Skipped annotations: 0
📝 Labels saved to: /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/yolo_scratch_only/labels/train/


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/annotations_unify/val2017_unify_scratch_only.json'

Merge Scratch and No-Damage Datasets

In this step, we define a function to **merge both `train` and `val` splits** of:

- Scratch images with **YOLO-formatted labels** (from CarDD)
- No-damage images with **empty or existing labels** (from Stanford Cars)

The function:
- Creates the required `images/train`, `labels/train`, `images/val`, and `labels/val` directories under a common output root.
- Copies scratch image-label pairs into their respective folders.
- Copies no-damage images and generates empty label files (if missing) to maintain YOLO compatibility.

In [ ]:
# define a function Merges both splits (train and val)
# it handles scratch images with YOLO labels and no_damage images with empty labels.


def merge_scratch_and_no_damage(
    scratch_image_dir, scratch_label_dir,
    nodamage_image_dir, nodamage_label_dir,
    output_root
):
    for split in ['train', 'val']:
        # Create output folders
        images_out = Path(output_root) / "images" / split
        labels_out = Path(output_root) / "labels" / split
        images_out.mkdir(parents=True, exist_ok=True)
        labels_out.mkdir(parents=True, exist_ok=True)

        # Merge scratch data
        print(f"Copying scratch data for {split}...")
        scratch_img_path = Path(scratch_image_dir) / split
        scratch_lbl_path = Path(scratch_label_dir) / split

        for img_file in scratch_img_path.glob("*.jpg"):
            shutil.copy(img_file, images_out / img_file.name)
        for lbl_file in scratch_lbl_path.glob("*.txt"):
            shutil.copy(lbl_file, labels_out / lbl_file.name)

        # Merge no_damage data
        print(f"Adding no_damage data for {split}...")
        nodmg_img_path = Path(nodamage_image_dir) / split
        nodmg_lbl_path = Path(nodamage_label_dir) / split

        for img_file in nodmg_img_path.glob("*.jpg"):
            shutil.copy(img_file, images_out / img_file.name)

        for img_file in nodmg_img_path.glob("*.jpg"):
            lbl_name = img_file.with_suffix(".txt").name
            lbl_file = nodmg_lbl_path / lbl_name

            if lbl_file.exists():
                shutil.copy(lbl_file, labels_out / lbl_name)
            else:
                # Create an empty label file if it doesn't exist
                (labels_out / lbl_name).write_text("")

        print(f"Merged {split} set completed.\n")





In [37]:
# merge the scratch data(carDD) and no scratch data(no damage)
merge_scratch_and_no_damage(
    scratch_image_dir="/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/yolo_scratch_only/images",
    scratch_label_dir="/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/yolo_scratch_only/labels",
    nodamage_image_dir="/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/images",
    nodamage_label_dir="/content/drive/MyDrive/vehicle_scratch_detection_v2/stanford_cars/no_damage_dataset/labels",
    output_root="/content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset"
)

📦 Copying scratch data for train...
➕ Adding no_damage data for train...
✅ Merged train set completed.

📦 Copying scratch data for val...
➕ Adding no_damage data for val...
✅ Merged val set completed.



## Data Validation
Verify YOLO Label Files Before Training

Before training your YOLOv8 model, it's critical to **validate all label files** to avoid errors during training.

This function checks all `.txt` annotation files in both `train` and `val` splits for:

-  Empty label files
-  Incorrect YOLO format (each line must have 5 components)
-  Invalid class IDs (must be `0` since we are using only the "scratch" class)
- Non-integer class IDs

In [38]:
# define a function to validate all label files before training
def verify_yolo_labels(labels_root):
    splits = ['train', 'val']
    issues = []

    for split in splits:
        label_dir = os.path.join(labels_root, split)
        print(f"\nVerifying {split} labels in: {label_dir}")
        count = 0

        for fname in os.listdir(label_dir):
            if not fname.endswith(".txt"):
                continue

            path = os.path.join(label_dir, fname)
            with open(path, "r") as f:
                lines = f.readlines()

            if not lines:
                issues.append((fname, "Empty file"))
                continue

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    issues.append((fname, f"Invalid YOLO format: {line.strip()}"))
                    continue

                try:
                    cls_id = int(parts[0])
                    if cls_id != 0:
                        issues.append((fname, f"Invalid class ID: {cls_id}"))
                except ValueError:
                    issues.append((fname, f"Non-integer class ID: {parts[0]}"))

            count += 1

        print(f" Checked {count} label files in '{split}'")

    if issues:
        print(f"\n Found {len(issues)} issues:")
        for fname, reason in issues[:10]:  # Show only first 10 for brevity
            print(f" - {fname}: {reason}")
    else:
        print("\n All labels are correctly formatted and valid.")


labels_root = "/content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels"
verify_yolo_labels(labels_root)



📁 Verifying train labels in: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/train
✅ Checked 2816 label files in 'train'

📁 Verifying val labels in: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/val
✅ Checked 810 label files in 'val'

❌ Found 2000 issues:
 - 1468.txt: Empty file
 - 1459.txt: Empty file
 - 1476.txt: Empty file
 - 1488.txt: Empty file
 - 1486.txt: Empty file
 - 1493.txt: Empty file
 - 1471.txt: Empty file
 - 1528.txt: Empty file
 - 1522.txt: Empty file
 - 1521.txt: Empty file


In [ ]:
# Run verification
verify_yolo_labels("/content/final_dataset/labels")


Count the Images and Labels

This step counts the number of image and label files in the train and val directories of the final YOLO-formatted dataset. It is a sanity-check that images and labels are correctly paired and that the dataset is properly split before training. Also verify balanced distribution between train and val sets.

In [33]:
# counts the number of image and label files in the train and val directories of the final YOLO-formatted dataset.

def count_images_and_labels(base_path):
    image_train = list(Path(base_path, "images/train").glob("*.[jp][pn]g"))
    image_val = list(Path(base_path, "images/val").glob("*.[jp][pn]g"))
    label_train = list(Path(base_path, "labels/train").glob("*.txt"))
    label_val = list(Path(base_path, "labels/val").glob("*.txt"))

    print(f" Image Count:")
    print(f"  ├─ Train images: {len(image_train)}")
    print(f"  └─ Val images:   {len(image_val)}")
    print(f"\n Label Count:")
    print(f"  ├─ Train labels: {len(label_train)}")
    print(f"  └─ Val labels:   {len(label_val)}")




In [ ]:
count_images_and_labels("/content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset")

## Model Training


Check PyTorch & CUDA Environment

In [1]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device")


Torch version: 2.6.0+cu124
CUDA available: True
CUDA device name: Tesla T4


 Load Pretrained YOLOv8 Model

 We use the pretrained YOLOv8s model as the base for transfer learning. ### 🔍 Model Choice: YOLOv8

We use **YOLOv8s** for training due to its balance of speed and accuracy. It is lightweight, supports custom datasets, and is ideal for real-time applications like scratch detection. The `ultralytics` library also makes training and deployment straightforward.


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # Loads the YOLOv8-small model


Train the Model
- data: Path to the data.yaml file.

- epochs: Number of training cycles.

- imgsz: Image size (640x640).

- batch: Batch size (adjust if GPU memory is limited).

- project: Output folder for training results.

In [2]:

model = YOLO("yolov8s.pt")

model.train(
    data="/content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="scratch_detector_yolov8s",
    project="/content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs"
)


Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=scratch_detector_yolov8s3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tru

train: Scanning /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/train... 4415 images, 1600 backgrounds, 1 corrupt: 100%|██████████| 4416/4416 [01:37<00:00, 45.10it/s] 

train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000001.jpg: 2 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000002.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000003.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000004.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000005.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000006.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000007.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/images/train/000008.jpg: 7 duplicate labels removed


train: New cache created: /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.4±0.4 MB/s, size: 276.9 KB)


val: Scanning /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/val... 1210 images, 400 backgrounds, 0 corrupt: 100%|██████████| 1210/1210 [02:05<00:00,  9.65it/s]


...

                   all       1210       1744      0.664      0.586      0.639      0.413



50 epochs completed in 2.226 hours.
Optimizer stripped from /content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs/scratch_detector_yolov8s3/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs/scratch_detector_yolov8s3/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs/scratch_detector_yolov8s3/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:21<00:00,  1.73it/s]


                   all       1210       1744      0.667      0.588      0.643      0.415
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs/scratch_detector_yolov8s3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dc1d0e40390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

Evaluate Model on Validation Set

After training, we load the best weights and evaluate on the validation split to get metrics like mAP, Precision, and Recall.

In [6]:
model = YOLO("/content/drive/MyDrive/vehicle_scratch_detection_v2/training_runs/scratch_detector_yolov8s3/weights/best.pt")
model.val()


Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 1.5±1.4 ms, read: 32.8±35.9 MB/s, size: 468.5 KB)


val: Scanning /content/drive/MyDrive/vehicle_scratch_detection_v2/final_dataset/labels/val.cache... 1210 images, 400 backgrounds, 0 corrupt: 100%|██████████| 1210/1210 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:34<00:00,  2.21it/s]


                   all       1210       1744      0.664      0.587      0.643      0.413
Speed: 0.5ms preprocess, 6.8ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val

image 1/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017/000012.jpg: 448x640 1 scratch, 73.1ms
image 2/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017/000015.jpg: 448x640 3 scratchs, 13.0ms
image 3/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017/000023.jpg: 448x640 1 scratch, 55.2ms
image 4/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017/000033.jpg: 640x448 2 scratchs, 139.3ms
image 5/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017/000040.jpg: 448x640 2 scratchs, 14.4ms
image 6/374 /content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_relea

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'scratch'}
 obb: None
 orig_img: array([[[246, 238, 231],
         [244, 238, 231],
         [243, 237, 232],
         ...,
         [ 37,  56,  64],
         [ 37,  62,  66],
         [ 35,  57,  63]],
 
        [[244, 238, 233],
         [244, 238, 233],
         [242, 238, 233],
         ...,
         [ 33,  52,  60],
         [ 32,  54,  60],
         [ 33,  55,  61]],
 
        [[243, 235, 228],
         [243, 235, 228],
         [243, 237, 230],
         ...,
         [ 32,  51,  58],
         [ 32,  54,  60],
         [ 33,  56,  64]],
 
        ...,
 
        [[191, 188, 184],
         [192, 189, 185],
         [193, 190, 186],
         ...,
         [147, 166, 163],
         [153, 172, 175],
         [159, 179, 180]],
 
        [[193, 190, 186],
         [195, 192, 188],
         [189, 186, 182],
         ...,
         [122, 13

Run Predictions on Test Set

We visually inspect predictions on unseen data. All predicted images with bounding boxes will be saved to the default runs/predict directory.



In [ ]:
model.predict(source="/content/drive/MyDrive/vehicle_scratch_detection_v2/CarDD/CarDD_release/CarDD_COCO/test2017", save=True, conf=0.25)


## Conclusion & Next Steps

### Key Findings
- The YOLOv8-based model successfully detects **vehicle scratches** with high precision on the validation set.
- The final dataset, constructed by merging scratch-only and no-damage samples, ensured balanced learning.
- Model training with `yolov8s` achieved strong detection performance while maintaining fast inference time.
- Label validation and image-label counting helped ensure high data quality before training.





### Limitations
- Some **scratch instances are hard to detect** in poor lighting or blurry images.
- **Imbalanced scratch sizes** (tiny vs. large) might have affected detection accuracy for edge cases.





###  Future Work
- **Modularization**: Refactor the notebook into a **production-grade ML pipeline** using modular Python scripts and configuration files.
- **Extend to multi-class detection** (e.g., deep scratch, dent, rust).
- Apply **advanced augmentations** to improve generalization.
- **Deployment Enhancements**: Deploy using **FastAPI or containerized Flask app** on cloud platforms (e.g., AWS, Azure).
- Export to ONNX for mobile integration.
- Explore **larger YOLO variants** (`YOLOv8m`, `YOLOv8l`, `YOLOv11`) and fine-tune on more diverse datasets.

